# Project Plan - Restaurant Recommendation System

This project is designed to develop a project plan for proposed software development for a group of restaurant owners in Marlborough, Massachussetts. It is described as a "consumer-focused recommendation system" that will consist of the following technologies:

- Alpine.js and Tailwind (Frontend)
- GraphQL (API)
- Go web and database server (Backend)
- Either PostgreSQL, EdgeDB, or PocketBase (Database)
- Cloud hosting (AWS, Azure, or GCP)

The softeare will ingest Yelp reviews of the participating restaurants. Restaurants will be updated monthly and Yelp reviews will be updated daily.

## Part 1 - Problem Setup

Using a provided template for the project plan with required steps, for each step, I have filled out the required team members and the number of hours. I have also provided the hourly rate for each team member role and thus am able to calculate the budget based on these estimates. The project plan also specifies best case scenario, expected scenario and worst case scenario.

The project plan is provided in the data folder.

### Uncertainties

- We have not been given a budget range by the client(s) so we are determining the budget based on a reasonable expectation for quality and budget
- This proposal does not assume any scope change and is based on the assumption that the client has a clear vision of the project. If the client changes the scope, the budget will need to be adjusted accordingly.
- This project has defined the number of hours (and thus the budget) but has not defined the wall time for the project and due dates. This will need to be determined in collaboration with the client and their needs.
- We will need to discuss the ongoing costs of hosting and maintenance with the client to determine the ongoing costs of the project post-launch.

### Additional Notes
- Several steps in the project can be done in parallel, which will help reduce the overall time to completion. These would be:
    - Developing pricing plan and developing implementation plan
    - Writing documentation and unit testing will be done in parallel with development
    - Developing marketing strategy and designing the brochure will be done in collaboration

## Step 2 - Model Specification 

An assumption provided is that all contributors to the project charge the same hourly rate. This means that less time for all members decreases the time by the same amount. This a minimization problem regarding cost but also a critical path analysis that will be determined by task dependency and overlap.

Normally we would have decision variables for individual team members but based on the instructions we will assume that all team members are interchangeable and thus the decision variables are the number of hours for each task and when they are scheduled. This way we can determine an optimal schedule.

## Decision Variables

- $x_{ij}$: Number of hours for task $i$ for team member $j$

# Step 3 - Programming

Below is the code to solve the optimization problem for the project plan. We will use the PuLP library to solve the linear programming problem. We will use the provided data to determine the optimal schedule for the project plan.

In [3]:
# First, defined the tasks and their durations based on the completed spreadsheet. I have chosen to use a list for the durations for best-case, expected, and worst-case scenarios.

# List of all tasks
tasks = ['A', 'B', 'C', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'E', 'F', 'G', 'H']

# Expected durations (in hours)
durations = {
    'A': [4,8,12],
    'B': [8,12,16],
    'C': [6,10,14],
    'D1': [12,16,20],
    'D2': [16,24,32],
    'D3': [16,24,32],
    'D4': [40,60,80],
    'D5': [8,12,16],
    'D6': [16,24,32],
    'D7': [16,24,32],
    'D8': [4,8,12],
    'E': [8,12,16],
    'F': [6,8,12],
    'G': [8,12,16],
    'H': [6,8,10]
}

# Immediate predecessors
predecessors = {
    'A': [],
    'B': [],
    'C': ['A'],
    'D1': ['A'],
    'D2': ['D1'],
    'D3': ['D1'],
    'D4': ['D2', 'D3'],
    'D5': ['D4'],
    'D6': ['D4'],
    'D7': ['D6'],
    'D8': ['D5', 'D7'],
    'E': ['B', 'C'],
    'F': ['D8', 'E'],
    'G': ['A', 'D8'],
    'H': ['F', 'G']
}

Now run the PuLP code to solve the optimization problem.

In [4]:
import pulp

# Scenario names and indices
scenarios = ['Best-Case', 'Expected', 'Worst-Case']
scenario_indices = [0, 1, 2]

# Uniform hourly rate (adjust as needed)
hourly_rate = 60  # dollars per hour

for scenario_name, idx in zip(scenarios, scenario_indices):
    print(f"--- {scenario_name} Scenario ---\n")
    
    # Extract durations for the current scenario
    scenario_durations = {task: durations[task][idx] for task in tasks}
    
    # Initialize the LP problem
    prob = pulp.LpProblem(f"Project_Scheduling_{scenario_name}", pulp.LpMinimize)
    
    # Define decision variables
    S = {task: pulp.LpVariable(f"S_{task}", lowBound=0) for task in tasks}
    T = pulp.LpVariable("T", lowBound=0)
    
    # Set the objective function
    prob += T, "Minimize_Project_Completion_Time"
    
    # Add constraints
    # 1. Project completion time constraints
    for task in tasks:
        prob += T >= S[task] + scenario_durations[task], f"Completion_Time_{task}"
    
    # 2. Precedence constraints
    for task, preds in predecessors.items():
        for pred in preds:
            prob += S[task] >= S[pred] + scenario_durations[pred], f"Precedence_{pred}_to_{task}"
    
    # Solve the LP problem
    prob.solve()
    
    # Display the status of the solution
    print(f"Status: {pulp.LpStatus[prob.status]}")
    
    if pulp.LpStatus[prob.status] == 'Optimal':
        # Display start times
        for task in tasks:
            print(f"Start time of {task}: {S[task].varValue} hours")
        
        # Display total project completion time
        print(f"Total project completion time: {T.varValue} hours")
        
        # Calculate total project cost
        total_hours = sum(scenario_durations.values())
        total_cost = hourly_rate * total_hours
        print(f"Total project cost: ${total_cost}")
        
        # Identify critical path
        print("\nTasks on the critical path:")
        for task in tasks:
            start_time = S[task].varValue
            finish_time = start_time + scenario_durations[task]
            slack = T.varValue - finish_time
            # Adjust for floating point precision
            if abs(slack) < 1e-6:
                print(f" - Task {task}")
    else:
        print("No optimal solution found.")
    
    print("\n" + "="*50 + "\n")

--- Best-Case Scenario ---

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/mattmiller/Sites/Decision Analytics - Assignment 2/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/6c2bd4e808d144b3b74bdbc56af65ef4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/4_/vbtv64ld2lv61jz5x9sljn5h0000gn/T/6c2bd4e808d144b3b74bdbc56af65ef4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 39 COLUMNS
At line 109 RHS
At line 144 BOUNDS
At line 145 ENDATA
Problem MODEL has 34 rows, 16 columns and 68 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-34) rows, 0 (-16) columns and 0 (-68) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 122
After Postsolve, objective 122, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 122 - 0 itera